# MoreLikeThis

`Elasticsearch` 中的 `More Like This （MLT）`是一种基于文本相似度的检索方法，它可以根据指定的文档或文本，查找与之相似的其他文档。More Like This 查询可以帮助用户发现与其感兴趣的内容类似的其他内容，从而提高搜索的精度和个性化。

`More Like This` 查询可以在 `Elasticsearch` 中使用，它的实现基于 Elasticsearch 的 Lucene 搜索引擎库。该查询需要指定要匹配的文档 ID 或文本内容，然后 Elasticsearch 会基于这个文档的词项生成一个查询，然后在索引中查找与查询相似的文档。

In [1]:
from elasticsearch_dsl import Document, Text, Keyword, Index, connections
from elasticsearch_dsl.query import MoreLikeThis

# 连接到Elasticsearch
connections.create_connection(hosts=['localhost'], timeout=20)

<Elasticsearch([{'host': 'localhost'}])>

In [2]:
# 创建索引并设置 mapping
index_name = 'index_cos'
index = Index(index_name)

if index.exists():
    index.delete()

index.settings(number_of_shards=1, number_of_replicas=0)

@index.document
class CosDocument(Document):
    title = Text(analyzer='snowball')
    body = Text(analyzer='snowball')
    
    class Index:
        name = index_name
    
# 向索引中添加数据
doc1 = CosDocument(title='Python programming', body='Python is a popular programming language.')
doc1.save()
doc2 = CosDocument(title='Java programming', body='Java is a widely used programming language.')
doc2.save()
doc3 = CosDocument(title='Java programming', body='I want a book.')
doc3.save()

'created'

In [4]:
# 计算相似度
query = MoreLikeThis(
    fields=['title', 'body'],
    like={'_id': doc1.meta.id},
    min_term_freq=1,    # 一篇文档中一个词语至少出现次数，小于这个值的词将被忽略，
    min_doc_freq=1,     # 一个词语最少在多少篇文档中出现，小于这个值的词会将被忽略
    max_query_terms=12  # 一条查询语句中允许最多查询词语的个数
)
search = CosDocument.search().query(query)
response = search.execute()
for hit in response:
    print('Similarity with doc1:', hit.meta.score)

Similarity with doc1: 1.5415431
Similarity with doc1: 0.28532696


- percent_terms_to_match：匹配项（term）的百分比，默认是0.3
- min_term_freq：一篇文档中一个词语至少出现次数，小于这个值的词将被忽略，默认是2
- max_query_terms：一条查询语句中允许最多查询词语的个数，默认是25
- stop_words：设置停止词，匹配时会忽略停止词
- min_doc_freq：一个词语最少在多少篇文档中出现，小于这个值的词会将被忽略，默认是无限制
- max_doc_freq：一个词语最多在多少篇文档中出现，大于这个值的词会将被忽略，默认是无限制
- min_word_len：最小的词语长度，默认是0
- max_word_len：最多的词语长度，默认无限制
- boost_terms：设置词语权重，默认是1
- boost：设置查询权重，默认是1

在上面的示例中，

1. 我们首先创建了一个`elasticsearch`的连接，
1. 然后使用elasticsearch_dsl创建了一个名为`index_cos`的索引，并定义了一个名为CosDocument的文档类型，其中包含了两个字段：title和body。
1. 接着，我们向索引中添加了几个文档，分别表示Python编程和Java编程。
1. 最后，我们使用MoreLikeThis查询计算了与doc1（表示Python编程）相似的文档，并打印出了它们的相似度分数。

> Snowball

Snowball（也称为Porter2）是一种流行的词干提取算法，用于将单词转换为它们的基本形式，以便它们可以更容易地与其他单词进行比较和分析。Snowball是Martin Porter开发的算法，它是Porter词干提取算法的改进版本。Snowball算法可以用于各种语言，包括英语、法语、德语、荷兰语、意大利语、葡萄牙语、西班牙语、瑞典语、挪威语、丹麦语、芬兰语、俄语、土耳其语等。

Snowball算法的基本思想是将单词转换为它们的基本形式，而不是仅仅削减词尾。例如，对于单词“running”，Snowball算法将其转换为“run”，而不是仅仅删除词尾“ing”。

Snowball算法的工作原理是将单词分解为一系列规则，然后应用这些规则以将单词转换为其基本形式。这些规则通常包括删除词尾、删除前缀、对元音字母进行转换等。Snowball算法还可以根据不同的语言和应用程序进行自定义配置。

在搜索引擎和信息检索中，Snowball算法通常用于对搜索查询进行处理，以便将查询中的单词转换为其基本形式，并与文档中的单词进行匹配。这样可以增加搜索的准确性和召回率，提高搜索结果的质量。

# Vector

> 使用`elasticsearch_dsl`存储向量数据并计算余弦相似度

使用`Elasticsearch`的`Vector`数据类型和`Vector Similarity`函数。下面是一个使用`elasticsearch_dsl`存储向量数据并计算余弦相似度的示例代码：

In [1]:
import numpy as np
from elasticsearch_dsl import Search, Document, Index, connections, DenseVector
from elasticsearch_dsl.query import ScriptScore, Q, MatchAll

# 连接到Elasticsearch
connections.create_connection(hosts=['localhost'], timeout=20)

# 创建索引并设置mapping，指定字段类型为DenseVector
dims = 50
index_name = 'index_vec'
index = Index(index_name)

if index.exists():
    index.delete()

@index.document
class VecDocument(Document):
    vector = DenseVector(dims=50)
    
    class Index:
        name = index_name

VecDocument.init()
# 向索引中添加向量数据
doc1 = VecDocument(meta={'id': '1'}, vector=np.random.random(dims).tolist())
doc1.save()
doc2 = VecDocument(meta={'id': '2'}, vector=np.random.random(dims).tolist())
doc2.save()

'created'

In [3]:
search = Search(index=index_name)
query_vector = np.random.random(dims).tolist()

# 计算余弦相似度
script = {
    "source": "cosineSimilarity(params.query_vector, 'vector') + 1.0",
    "params": {"query_vector": query_vector}
}
query = MatchAll()
script_score = ScriptScore(query=query, script=script)
s = search.query(script_score)

index.refresh()
response = s.execute()
for hit in response:
    print('Cosine similarity with query vector:', hit.meta.score)

Cosine similarity with query vector: 1.7647238
Cosine similarity with query vector: 1.728144


1. 首先使用`elasticsearch_dsl`创建了一个名为`index_vec`的索引，定义了一个名为`VecDocument`的文档类型，其中包含了一个名为`vector`的向量字段。
1. 接着，我们向索引中添加了两个带有向量数据的文档，分别表示两个向量。
1. 最后，我们使用`ScriptScore`查询计算了与查询向量的余弦相似度，并打印出了它们的分数。

在这个查询中，我们使用了`cosineSimilarity()`函数，它计算两个向量之间的余弦相似度，其中第一个参数是查询向量，第二个参数是文档中的向量字段。注意，`ScriptScore`函数会将所有分数加上`1.0`，以避免出现负数分数。

# 使用Transformers模型进行向量化

In [ ]:
from functools import lru_cache
from transformers import BertTokenizer, ErnieModel


@lru_cache()
def vec_model(model_path):
    """"""
    tokenizer = BertTokenizer.from_pretrained(model_path)
    model = ErnieModel.from_pretrained(model_path)
    model.eval()
    return model, tokenizer

def text2vec(text, model=None, tokenizer=None, dtype='list', flatten=True):
    """"""
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    vec = output[1]
    if flatten:
        vec = vec.flatten()
    if dtype == 'list':
        vec = vec.tolist()
    return vec

In [ ]:
model_path = '/data/models/ernie-3.0-base-zh'
model, tokenizer = vec_model(model_path)

# 需要预测的文本
text = "这是一个测试文本"

text_vec = text2vec(text, model, tokenizer)

In [ ]:
len(text_vec)

In [ ]:
print(f"Vec: {text_vec[:10]}")

------